# Установка библиотек  
после успешной установки, строки комментим

In [ ]:
# Установим необходимые библиотеки
!pip install tensorflow opencv-python-headless numpy scikit-learn
!pip install joblib
!pip install numpy opencv-python-headless
!pip install h5py
!pip install  gdal
!pip install matplotlib



In [11]:
!pip install gdal

  Using cached GDAL-3.9.0.tar.gz (839 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build gdal


  error: subprocess-exited-with-error
  
  × Building wheel for gdal (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [156 lines of output]
      Using numpy 2.0.0
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\osgeo
      copying osgeo\gdal.py -> build\lib.win-amd64-cpython-311\osgeo
      copying osgeo\gdalconst.py -> build\lib.win-amd64-cpython-311\osgeo
      copying osgeo\gdalnumeric.py -> build\lib.win-amd64-cpython-311\osgeo
      copying osgeo\gdal_array.py -> build\lib.win-amd64-cpython-311\osgeo
      copying osgeo\gnm.py -> build\lib.win-amd64-cpython-311\osgeo
      copying osgeo\ogr.py -> build\lib.win-amd64-cpython-311\osgeo
      copying osgeo\osr.py -> build\lib.win-amd64-cpython-311\osgeo
      copying osgeo\__init__.py -> build\lib.win-amd64-cpython-311\osgeo
      creating build\lib.win-amd64-cpython-311\osgeo_utils

# Импорт необходимых модулей

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
import joblib
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal, osr
import json

In [ ]:
#import os
#os.environ['USE_PATH_FOR_GDAL_PYTHON'] = 'YES'

#from osgeo import gdal, osr


# Константы

In [ ]:
# Название папки с изображениями - кропами
crops_folder = '1_20'

# Название папки с файлами подложек
template_folder = 'layouts'


# Определение номера подложки для каждого файла кропа

In [ ]:
# Загрузка предобученной модели VGG16
base_model = VGG16(weights='imagenet', include_top=True)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)

# Функция для извлечения признаков
def extract_features(image_path, model):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Изменение размера изображения до 224x224
    image = preprocess_input(image)  # Предобработка изображения для VGG16
    features = model.predict(np.expand_dims(image, axis=0))  # Извлечение признаков
    return features.flatten()


## Считывание путей к файлам

In [ ]:
# Определение корневой папки проекта
root_folder = os.getcwd()

# Полные пути к папкам
crops_path = os.path.join(root_folder, crops_folder)
template_path = os.path.join(root_folder, template_folder)

# Получение списка файлов в папках
crops_files = [os.path.join(crops_path, f) for f in os.listdir(crops_path) if os.path.isfile(os.path.join(crops_path, f))]
template_files = [os.path.join(template_path, f) for f in os.listdir(template_path) if os.path.isfile(os.path.join(template_path, f))]

# Печать количества файлов
print(f"Количество кропов в папке '{crops_folder}': {len(crops_files)}")
print(f"Количество подложек в папке '{template_folder}': {len(template_files)}")

# Печать путей к файлам
print("\nПути к изображениям:")
for path in crops_files:
    print(path)

print("\nПути к подложкам:")
for path in template_files:
    print(path)


## Подготовка модуля ИИ  
для определения номера подложки

In [ ]:
# Создание меток для обучения из списка файлов подложек
labels = list(range(len(template_files)))

In [ ]:
labels

In [ ]:
# Извлечение признаков для всех изображений
image_features = [extract_features(path, model) for path in crops_files]

# Извлечение признаков для всех подложек
template_features = [extract_features(path, model) for path in template_files]

# Создание и обучение стандартного скейлера
scaler = StandardScaler()
image_features = scaler.fit_transform(image_features)
template_features = scaler.transform(template_features)

# Подготовка данных для обучения
X = template_features
y = labels

# Разделение данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели SVM
classifier = SVC(kernel='linear')
classifier.fit(X_train, y_train)

# Проверка точности модели на тестовой выборке
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy * 100:.2f}%')

## Сохраним модель на диск

In [ ]:
# Сохранение модели и скейлера
joblib.dump(classifier, os.path.join(root_folder, 'svm_classifier.joblib'))
joblib.dump(scaler, os.path.join(root_folder, 'scaler.joblib'))

## Загрузка сохранённой модели

In [ ]:
# Определение корневой папки проекта
root_folder = os.getcwd()

# Загрузка модели и скейлера
classifier = joblib.load(os.path.join(root_folder, 'svm_classifier.joblib'))
scaler = joblib.load(os.path.join(root_folder, 'scaler.joblib'))

## Функция определения подложки  
с примера использования

In [ ]:
# Функция для предсказания подложки
def predict_template(image_path, model, scaler, classifier):
    features = extract_features(image_path, model)
    features = scaler.transform([features])
    template_index = classifier.predict(features)[0]
    return template_index

# Путь к новому изображению для теста
test_image_path = r'D:\18\1_20\crop_3_3_0000.tif'

# Предсказание подложки
predicted_template_index = predict_template(test_image_path, model, scaler, classifier)
print(f'Предсказанный индекс подложки: {predicted_template_index}')

In [ ]:
# файл кропа берём из списка
test_image_path = crops_files[5]
# Предсказание подложки
predicted_template_index = predict_template(test_image_path, model, scaler, classifier)
print(f'Предсказанный индекс подложки: {predicted_template_index}')


## Выведем подложки для всех файлов кропов

In [ ]:
# Предсказание подложки для всех элементов списка crops_files
predicted_indices = []
for test_image_path in crops_files:
    predicted_template_index = predict_template(test_image_path, model, scaler, classifier)
    predicted_indices.append(predicted_template_index)
   # print(f'Путь к изображению: {test_image_path} -> Предсказанный индекс подложки: {predicted_template_index}')

# Вывод всех предсказанных индексов
print("\nВсе предсказанные индексы подложек:")
for i, predicted_index in enumerate(predicted_indices):
    print(f'Кроп {i}: {predicted_index}')

### да. постановщик задачи говорил, что все кропы нам даны с одной подложки

# Определение координат кропа

## Реcемплинг и выравнивание изображений

In [ ]:
# Открываем изображение и референсное изображение
image_path = "D:\\18\\1_20\\crop_1_0_0000.tif"
reference_path = "D:\\18\\layouts\\layout_2021-08-16.tif"

In [ ]:
# Открываем исходное изображение и считываем его данные
image = gdal.Open(image_path)
reference = gdal.Open(reference_path)

In [ ]:
# Получаем информацию о проекциях
image_proj = osr.SpatialReference(wkt=image.GetProjection())
reference_proj = osr.SpatialReference(wkt=reference.GetProjection())

In [ ]:
# Трансформируем координаты углов исходного изображения в систему координат EPSG:32637
transform = osr.CoordinateTransformation(reference_proj, image_proj)

### Функция извлечения геокоординат 

In [ ]:
def get_corner_coords(dataset):
    geo_transform = dataset.GetGeoTransform()
    x_min = geo_transform[0]
    x_max = x_min + geo_transform[1] * dataset.RasterXSize
    y_min = geo_transform[3] + geo_transform[5] * dataset.RasterYSize
    y_max = geo_transform[3]

    return [(x_min, y_max), (x_max, y_max), (x_max, y_min), (x_min, y_min)]

### Извлекаем геокоординаты и трансформируем

In [ ]:
corners = get_corner_coords(reference)
corners_transformed = [transform.TransformPoint(corner[0], corner[1]) for corner in corners]


In [ ]:
# Записываем координаты в текстовый файл
with open("corners_coordinates.txt", "w") as file:
    for corner in corners_transformed:
        file.write(f"{corner[0]:.3f}; {corner[1]:.3f}\n")

In [ ]:
# Создаем GeoJSON

geojson_data = {
    "type": "Polygon",
    "coordinates": [[
        [corner[0], corner[1]] for corner in corners_transformed
    ]]
}

with open("corners_coordinates.geojson", "w") as file:
    json.dump(geojson_data, file)

In [ ]:
# Перепроецируем изображение в EPSG:32637
output_path = "output_image.tif"
gdal.Warp(output_path, image, dstSRS="EPSG:32637")

In [ ]:
type(image)

In [ ]:
type(gdal.Warp(output_path, image, dstSRS="EPSG:32637"))

### визуализируем изображения

In [ ]:
# исходное


# Открываем изображение
image_path = "D:\\18\\1_20\\crop_1_0_0000.tif"
image = gdal.Open(image_path)

# Проверка типа объекта
print(type(image))  # <class 'osgeo.gdal.Dataset'>

# Считывание данных из каналов
r_band = image.GetRasterBand(1).ReadAsArray()
g_band = image.GetRasterBand(2).ReadAsArray()
b_band = image.GetRasterBand(3).ReadAsArray()
nir_band = image.GetRasterBand(4).ReadAsArray()

# Нормализация данных для визуализации
r_band = (r_band - r_band.min()) / (r_band.max() - r_band.min())
g_band = (g_band - g_band.min()) / (g_band.max() - g_band.min())
b_band = (b_band - b_band.min()) / (b_band.max() - b_band.min())
nir_band = (nir_band - nir_band.min()) / (nir_band.max() - nir_band.min())

# Создание композитного изображения RGB
rgb_image = np.dstack((r_band, g_band, b_band))

# Визуализация каналов и композитного изображения
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0, 0].imshow(r_band, cmap='Reds')
axs[0, 0].set_title('Red Channel')
axs[0, 0].axis('off')

axs[0, 1].imshow(g_band, cmap='Greens')
axs[0, 1].set_title('Green Channel')
axs[0, 1].axis('off')

axs[1, 0].imshow(b_band, cmap='Blues')
axs[1, 0].set_title('Blue Channel')
axs[1, 0].axis('off')

axs[1, 1].imshow(rgb_image)
axs[1, 1].set_title('RGB Composite')
axs[1, 1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# трансформированное
image = gdal.Warp(output_path, image, dstSRS="EPSG:32637")

# Проверка типа объекта
print(type(image))  # <class 'osgeo.gdal.Dataset'>

# Считывание данных из каналов
r_band = image.GetRasterBand(1).ReadAsArray()
g_band = image.GetRasterBand(2).ReadAsArray()
b_band = image.GetRasterBand(3).ReadAsArray()
nir_band = image.GetRasterBand(4).ReadAsArray()

# Нормализация данных для визуализации
r_band = (r_band - r_band.min()) / (r_band.max() - r_band.min())
g_band = (g_band - g_band.min()) / (g_band.max() - g_band.min())
b_band = (b_band - b_band.min()) / (b_band.max() - b_band.min())
nir_band = (nir_band - nir_band.min()) / (nir_band.max() - nir_band.min())

# Создание композитного изображения RGB
rgb_image = np.dstack((r_band, g_band, b_band))

# Визуализация каналов и композитного изображения
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0, 0].imshow(r_band, cmap='Reds')
axs[0, 0].set_title('Red Channel')
axs[0, 0].axis('off')

axs[0, 1].imshow(g_band, cmap='Greens')
axs[0, 1].set_title('Green Channel')
axs[0, 1].axis('off')

axs[1, 0].imshow(b_band, cmap='Blues')
axs[1, 0].set_title('Blue Channel')
axs[1, 0].axis('off')

axs[1, 1].imshow(rgb_image)
axs[1, 1].set_title('RGB Composite')
axs[1, 1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# визуализация подложек

image= gdal.Open(template_files[0])

# Проверка типа объекта
print(type(image))  # <class 'osgeo.gdal.Dataset'>

# Считывание данных из каналов
r_band = image.GetRasterBand(1).ReadAsArray()
g_band = image.GetRasterBand(2).ReadAsArray()
b_band = image.GetRasterBand(3).ReadAsArray()
nir_band = image.GetRasterBand(4).ReadAsArray()

# Нормализация данных для визуализации
r_band = (r_band - r_band.min()) / (r_band.max() - r_band.min())
g_band = (g_band - g_band.min()) / (g_band.max() - g_band.min())
b_band = (b_band - b_band.min()) / (b_band.max() - b_band.min())
nir_band = (nir_band - nir_band.min()) / (nir_band.max() - nir_band.min())

# Создание композитного изображения RGB
rgb_image = np.dstack((r_band, g_band, b_band))

# Визуализация каналов и композитного изображения
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0, 0].imshow(r_band, cmap='Reds')
axs[0, 0].set_title('Red Channel')
axs[0, 0].axis('off')

axs[0, 1].imshow(g_band, cmap='Greens')
axs[0, 1].set_title('Green Channel')
axs[0, 1].axis('off')

axs[1, 0].imshow(b_band, cmap='Blues')
axs[1, 0].set_title('Blue Channel')
axs[1, 0].axis('off')

axs[1, 1].imshow(rgb_image)
axs[1, 1].set_title('RGB Composite')
axs[1, 1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# визуализация подложек

image = gdal.Open(template_files[1])

# Проверка типа объекта
print(type(image))  # <class 'osgeo.gdal.Dataset'>

# Считывание данных из каналов
r_band = image.GetRasterBand(1).ReadAsArray()
g_band = image.GetRasterBand(2).ReadAsArray()
b_band = image.GetRasterBand(3).ReadAsArray()
nir_band = image.GetRasterBand(4).ReadAsArray()

# Нормализация данных для визуализации
r_band = (r_band - r_band.min()) / (r_band.max() - r_band.min())
g_band = (g_band - g_band.min()) / (g_band.max() - g_band.min())
b_band = (b_band - b_band.min()) / (b_band.max() - b_band.min())
nir_band = (nir_band - nir_band.min()) / (nir_band.max() - nir_band.min())

# Создание композитного изображения RGB
rgb_image = np.dstack((r_band, g_band, b_band))

# Визуализация каналов и композитного изображения
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0, 0].imshow(r_band, cmap='Reds')
axs[0, 0].set_title('Red Channel')
axs[0, 0].axis('off')

axs[0, 1].imshow(g_band, cmap='Greens')
axs[0, 1].set_title('Green Channel')
axs[0, 1].axis('off')

axs[1, 0].imshow(b_band, cmap='Blues')
axs[1, 0].set_title('Blue Channel')
axs[1, 0].axis('off')

axs[1, 1].imshow(rgb_image)
axs[1, 1].set_title('RGB Composite')
axs[1, 1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# визуализация подложек

image= gdal.Open(template_files[2])

# Проверка типа объекта
print(type(image))  # <class 'osgeo.gdal.Dataset'>

# Считывание данных из каналов
r_band = image.GetRasterBand(1).ReadAsArray()
g_band = image.GetRasterBand(2).ReadAsArray()
b_band = image.GetRasterBand(3).ReadAsArray()
nir_band = image.GetRasterBand(4).ReadAsArray()

# Нормализация данных для визуализации
r_band = (r_band - r_band.min()) / (r_band.max() - r_band.min())
g_band = (g_band - g_band.min()) / (g_band.max() - g_band.min())
b_band = (b_band - b_band.min()) / (b_band.max() - b_band.min())
nir_band = (nir_band - nir_band.min()) / (nir_band.max() - nir_band.min())

# Создание композитного изображения RGB
rgb_image = np.dstack((r_band, g_band, b_band))

# Визуализация каналов и композитного изображения
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0, 0].imshow(r_band, cmap='Reds')
axs[0, 0].set_title('Red Channel')
axs[0, 0].axis('off')

axs[0, 1].imshow(g_band, cmap='Greens')
axs[0, 1].set_title('Green Channel')
axs[0, 1].axis('off')

axs[1, 0].imshow(b_band, cmap='Blues')
axs[1, 0].set_title('Blue Channel')
axs[1, 0].axis('off')

axs[1, 1].imshow(rgb_image)
axs[1, 1].set_title('RGB Composite')
axs[1, 1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# визуализация подложек

image= gdal.Open(template_files[3])

# Проверка типа объекта
print(type(image))  # <class 'osgeo.gdal.Dataset'>

# Считывание данных из каналов
r_band = image.GetRasterBand(1).ReadAsArray()
g_band = image.GetRasterBand(2).ReadAsArray()
b_band = image.GetRasterBand(3).ReadAsArray()
nir_band = image.GetRasterBand(4).ReadAsArray()

# Нормализация данных для визуализации
r_band = (r_band - r_band.min()) / (r_band.max() - r_band.min())
g_band = (g_band - g_band.min()) / (g_band.max() - g_band.min())
b_band = (b_band - b_band.min()) / (b_band.max() - b_band.min())
nir_band = (nir_band - nir_band.min()) / (nir_band.max() - nir_band.min())

# Создание композитного изображения RGB
rgb_image = np.dstack((r_band, g_band, b_band))

# Визуализация каналов и композитного изображения
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0, 0].imshow(r_band, cmap='Reds')
axs[0, 0].set_title('Red Channel')
axs[0, 0].axis('off')

axs[0, 1].imshow(g_band, cmap='Greens')
axs[0, 1].set_title('Green Channel')
axs[0, 1].axis('off')

axs[1, 0].imshow(b_band, cmap='Blues')
axs[1, 0].set_title('Blue Channel')
axs[1, 0].axis('off')

axs[1, 1].imshow(rgb_image)
axs[1, 1].set_title('RGB Composite')
axs[1, 1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# визуализация кропов

image = gdal.Open(crops_files[0])

# Проверка типа объекта
print(type(image))  # <class 'osgeo.gdal.Dataset'>

# Считывание данных из каналов
r_band = image.GetRasterBand(1).ReadAsArray()
g_band = image.GetRasterBand(2).ReadAsArray()
b_band = image.GetRasterBand(3).ReadAsArray()
nir_band = image.GetRasterBand(4).ReadAsArray()

# Нормализация данных для визуализации
r_band = (r_band - r_band.min()) / (r_band.max() - r_band.min())
g_band = (g_band - g_band.min()) / (g_band.max() - g_band.min())
b_band = (b_band - b_band.min()) / (b_band.max() - b_band.min())
nir_band = (nir_band - nir_band.min()) / (nir_band.max() - nir_band.min())

# Создание композитного изображения RGB
rgb_image = np.dstack((r_band, g_band, b_band))

# Визуализация каналов и композитного изображения
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0, 0].imshow(r_band, cmap='Reds')
axs[0, 0].set_title('Red Channel')
axs[0, 0].axis('off')

axs[0, 1].imshow(g_band, cmap='Greens')
axs[0, 1].set_title('Green Channel')
axs[0, 1].axis('off')

axs[1, 0].imshow(b_band, cmap='Blues')
axs[1, 0].set_title('Blue Channel')
axs[1, 0].axis('off')

axs[1, 1].imshow(rgb_image)
axs[1, 1].set_title('RGB Composite')
axs[1, 1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# визуализация кропов

image = gdal.Open(crops_files[9])

# Проверка типа объекта
print(type(image))  # <class 'osgeo.gdal.Dataset'>

# Считывание данных из каналов
r_band = image.GetRasterBand(1).ReadAsArray()
g_band = image.GetRasterBand(2).ReadAsArray()
b_band = image.GetRasterBand(3).ReadAsArray()
nir_band = image.GetRasterBand(4).ReadAsArray()

# Нормализация данных для визуализации
r_band = (r_band - r_band.min()) / (r_band.max() - r_band.min())
g_band = (g_band - g_band.min()) / (g_band.max() - g_band.min())
b_band = (b_band - b_band.min()) / (b_band.max() - b_band.min())
nir_band = (nir_band - nir_band.min()) / (nir_band.max() - nir_band.min())

# Создание композитного изображения RGB
rgb_image = np.dstack((r_band, g_band, b_band))

# Визуализация каналов и композитного изображения
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0, 0].imshow(r_band, cmap='Reds')
axs[0, 0].set_title('Red Channel')
axs[0, 0].axis('off')

axs[0, 1].imshow(g_band, cmap='Greens')
axs[0, 1].set_title('Green Channel')
axs[0, 1].axis('off')

axs[1, 0].imshow(b_band, cmap='Blues')
axs[1, 0].set_title('Blue Channel')
axs[1, 0].axis('off')

axs[1, 1].imshow(rgb_image)
axs[1, 1].set_title('RGB Composite')
axs[1, 1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# визуализация кропов

image = gdal.Open(crops_files[0])

# Проверка типа объекта
print(type(image))  # <class 'osgeo.gdal.Dataset'>

# Считывание данных из каналов
r_band = image.GetRasterBand(1).ReadAsArray()
g_band = image.GetRasterBand(2).ReadAsArray()
b_band = image.GetRasterBand(3).ReadAsArray()
nir_band = image.GetRasterBand(4).ReadAsArray()

# Нормализация данных для визуализации
r_band = (r_band - r_band.min()) / (r_band.max() - r_band.min())
g_band = (g_band - g_band.min()) / (g_band.max() - g_band.min())
b_band = (b_band - b_band.min()) / (b_band.max() - b_band.min())
nir_band = (nir_band - nir_band.min()) / (nir_band.max() - nir_band.min())

# Создание композитного изображения RGB
rgb_image = np.dstack((r_band, g_band, b_band))

# Визуализация каналов и композитного изображения
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0, 0].imshow(r_band, cmap='Reds')
axs[0, 0].set_title('Red Channel')
axs[0, 0].axis('off')

axs[0, 1].imshow(g_band, cmap='Greens')
axs[0, 1].set_title('Green Channel')
axs[0, 1].axis('off')

axs[1, 0].imshow(b_band, cmap='Blues')
axs[1, 0].set_title('Blue Channel')
axs[1, 0].axis('off')

axs[1, 1].imshow(rgb_image)
axs[1, 1].set_title('RGB Composite')
axs[1, 1].axis('off')

plt.tight_layout()
plt.show()


## Совмещение изображений


## Вычисление координат углов

## Конвертация координат

## Сохранение результатов